# Question Answering –  Fine-tuning – BERT-base


Ce notebook entraîne un modèle de Question Answering extractif
sur les données SQuAD préprocessées.


## Objectifs

- Charger les données préprocessées
- Charger un modèle QA pré-entraîné
- Fine-tuner avec Trainer
- Sauvegarder le meilleur modèle


In [ ]:
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)
import numpy as np


: 

## Chargement des données préprocessées

Les données ont été prétraitées dans le notebook précédent et sont
chargées directement depuis le disque.


In [ ]:
tokenized_datasets = load_from_disk("outputs/tokenized_squad")

## Data collator

Permet de créer les batchs correctement.


In [ ]:
data_collator = DefaultDataCollator()


: 

## Tokenizer

Le tokenizer associé au modèle est chargé pour assurer la cohérence
du pipeline d’entraînement et d’inférence.


In [ ]:
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


## Modèle

On charge un modèle compatible Question Answering.


In [ ]:
model_checkpoint = "bert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


## Paramètres d’entraînement


In [ ]:
training_args = TrainingArguments(
    output_dir="outputs/checkpoints/bert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="outputs/logs/bert",
    logging_steps=50,
    save_total_limit=1,
    load_best_model_at_end=True
)


## Entraînement

On lance le fine-tuning avec Trainer.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [ ]:
trainer.train()


## Sauvegarde

In [ ]:
trainer.save_model("outputs/checkpoints/bert/final")
tokenizer.save_pretrained("outputs/checkpoints/bert/final")


## Conclusion

Le modèle a été fine-tuné et sauvegardé.
La prochaine étape consistera à évaluer la qualité (EM/F1) et la latence,
puis à comparer 3 modèles.
